In [45]:
from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_community.callbacks import get_openai_callback
import os
import pandas as pd
import json
import traceback
from dotenv import load_dotenv
import PyPDF2

In [46]:
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0.7)

In [47]:
def openFile(fileName:str):
    file_path = os.path.join(os.pardir, fileName)

    try:
        with open(file_path, 'r') as file:
            data = file.read()
            return data
    except FileNotFoundError:
        print(f"The file {fileName} was not found in the project's main folder.")

In [48]:
RESPONSE_JSON = openFile("response.json")
print(RESPONSE_JSON)

{
    "1":{
        "no":1,
        "mcq":"mutiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2":{
        "no":2,
        "mcq":"mutiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3":{
        "no":3,
        "mcq":"mutiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}


In [5]:
template = """
Text: {text}
You are an expert MCQ maker, Given the above text, it is your job to create a quiz
of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and all the questions are confirming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs.
###RESPONSE_JSON
{response_json}
"""

In [6]:
quiz_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template= template
    )

In [7]:
quiz_chain = LLMChain(
    llm = llm,
    prompt = quiz_prompt,
    output_key = "quiz",
    verbose = True
)

In [8]:
eval_template = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students, you need to 
evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
If the quiz is not at par with the cognitive and analytical abilities of the students, update the quiz questions  
which needs to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}

Analysis from an expert English Writer of the above quiz:
"""

In [9]:
eval_prompt = PromptTemplate(
    input_variables= ["quiz","subject"],
    template= eval_template
)

In [10]:
eval_chain = LLMChain(
    llm = llm,
    prompt = eval_prompt,
    output_key = "review",
    verbose = True
)

In [11]:
chain = SequentialChain(chains=[quiz_chain,eval_chain],
                        input_variables = ["text","number","subject","tone","response_json"],
                        output_variables = ["quiz","review"] ,
                        verbose = True)

In [21]:
data = openFile("text.txt")
print(data)

Elon Reeve Musk (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$190 billion as of March 2024, according to the Bloomberg Billionaires Index, and $195 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]

A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania, and rec

In [23]:
args = {
    "text":data,
    "number":4,
    "subject":"Elon Musk",
    "tone":"simple",
    "response_json": RESPONSE_JSON
}

In [24]:
with get_openai_callback() as cb:
    res = chain(args)

/Users/prateek/Documents/GitHub/MCQ_GenAI/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Elon Reeve Musk (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$190 billion as of March 2024, according to the Bloomberg Billionaires Index, and $195 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]

A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated a

In [42]:
# token and cost details of the chain

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1765
Prompt Tokens:1420
Completion Tokens:345
Total Cost:0.0028199999999999996


In [25]:
print(res)

{'text': "Elon Reeve Musk (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$190 billion as of March 2024, according to the Bloomberg Billionaires Index, and $195 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]\n\nA member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylva

In [41]:
quiz = res.get('quiz')
quiz = json.loads(quiz)
type(quiz)

dict

In [43]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

quizDF = pd.DataFrame(quiz_table_data)

In [44]:
quizDF

,MCQ,Choices,Correct
0,What is Elon Musk's primary role at SpaceX?,a: Founder | b: CEO | c: CTO | d: Chairman,a
1,Which company did Elon Musk co-found with his ...,a: PayPal | b: Tesla | c: SpaceX | d: Zip2,d
2,What controversial views has Elon Musk been cr...,a: COVID-19 misinformation | b: Antisemitic co...,d
3,Which company did Elon Musk acquire for $44 bi...,a: Tesla | b: SpaceX | c: Twitter | d: PayPal,c
